In [10]:
!pip install requests plotly gradio opencv-python kaleido condacolab ultralytics

  Using cached gradio-5.18.0-py3-none-any.whl.metadata (16 kB)
  Using cached condacolab-0.1.10-py3-none-any.whl.metadata (5.5 kB)
  Using cached ultralytics-8.3.78-py3-none-any.whl.metadata (35 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached fastapi-0.115.8-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.5.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached gradio_client-1.7.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached MarkupSafe-2.1.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached ruff-0.9.7-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (25 kB)
  Using cached safehttpx-0.1.6-py3-none-any.whl.metadata (4.2 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached starlette-0.46.0-py3-none-any.

In [2]:
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:22
🔁 Restarting kernel...


In [3]:
!conda install -c conda-forge python-kaleido

Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - python-kaleido


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.1.31  |       hbcca054_0         154 KB  conda-forge
    certifi-2025.1.31          |     pyhd8ed1ab_0         159 KB  conda-forge
    conda-24.11.3              |  py311h38be061_0         1.1 MB  conda-forge
    expat-2.6.4                |       h5888daf_0         135 KB  conda-forge
    font-ttf-dejavu-sans-mono-2.37|       hab24e00_0         388 KB  conda-forge
    font-ttf-inconsolata-3.000 |       h77eed37_0          94 KB  conda-forge
    font-ttf-source-code-pro-2.038|       h77eed37_0         684 KB  conda-forge
    font-ttf-ubuntu-0.83       |       h77eed37_3         1.5 MB  conda-forge
    fontconfig-2.15.0          |       

In [1]:
import requests
import pandas as pd
import plotly.graph_objects as go
from ultralytics import YOLO
import cv2
import time
import gradio as gr

In [2]:
API_KEY = "ITWJ6NDTF45CBTDO"

In [3]:
def get_stock_candlestick_data(symbol, interval="5min", output_size="compact"):
    """
    Fetch stock candlestick data from Alpha Vantage.
    """
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&apikey={API_KEY}&outputsize={output_size}"
    print(f"Fetching data from: {url}")
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        print("API Response:", data)
        if f"Time Series ({interval})" in data:
            return data[f"Time Series ({interval})"]
        else:
            print("Error: No candlestick data found in response.")
            print(data)
            return None
    else:
        print(f"Error fetching data: {response.status_code}")
        print(response.text)
        return None

In [4]:
def process_stock_candlestick_data(data):
    """
    Process Alpha Vantage stock candlestick data into a DataFrame.
    """
    rows = []
    for timestamp, values in data.items():
        rows.append({
            "timestamp": timestamp,
            "open": float(values["1. open"]),
            "high": float(values["2. high"]),
            "low": float(values["3. low"]),
            "close": float(values["4. close"]),
            "volume": float(values["5. volume"])
        })
    return pd.DataFrame(rows)

In [5]:
def generate_candlestick_chart(df):
    """
    Generate a candlestick chart using Plotly.
    """
    fig = go.Figure(data=[go.Candlestick(
        x=df["timestamp"],
        open=df["open"],
        high=df["high"],
        low=df["low"],
        close=df["close"]
    )])
    fig.update_layout(
        title="Candlestick Chart",
        xaxis_title="Time",
        yaxis_title="Price",
        xaxis_rangeslider_visible=False
    )
    fig.show()
    fig.write_image("candlestick.png")

In [6]:
def yolo_model(img_path):
    model = YOLO("/content/best.pt")
    results = model(img_path)

    gap_up_count = 0
    gap_down_count = 0

    for result in results:
        classes = result.boxes.cls
        for cls in classes:
            if cls == 0:
                gap_down_count += 1
            elif cls == 1:
                gap_up_count += 1
        annotated_image = result.plot()

    return annotated_image, gap_up_count, gap_down_count

In [7]:
def detect_gap_patterns(symbol):
    """
    Main function to fetch data, generate charts, and detect GAP patterns in real-time.
    """
    while True:
        data = get_stock_candlestick_data(symbol)
        if not data:
            return "Error: Failed to fetch stock data. Please check the stock symbol or try again later."
        df = process_stock_candlestick_data(data)
        generate_candlestick_chart(df)
        annotated_image, gap_up_count, gap_down_count = yolo_model("candlestick.png")
        cv2.imwrite("annotated_output.png", annotated_image)
        yield "annotated_output.png", gap_up_count, gap_down_count
        time.sleep(5)

In [13]:
with gr.Blocks() as demo:
    gr.Markdown("# GAP Pattern Detection in Real-Time Stock Charts")
    gr.Markdown("Enter a stock symbol (e.g., AAPL) to detect GAP UP and GAP DOWN patterns in real-time candlestick charts.")
    with gr.Row():
        symbol_input = gr.Textbox(label="Stock Symbol", placeholder="Enter a stock symbol (e.g., AAPL)")
        submit_button = gr.Button("Start Real-Time Detection")
    with gr.Row():
        output_image = gr.Image(label="Annotated Candlestick Chart")
        gap_up_output = gr.Textbox(label="GAP UP Count")
        gap_down_output = gr.Textbox(label="GAP DOWN Count")
    submit_button.click(
        fn=detect_gap_patterns,
        inputs=symbol_input,
        outputs=[output_image, gap_up_output, gap_down_output],
        #live=True
    )
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a4932569c87b7f2da4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Fetching data from: https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=AAPL&interval=5min&apikey=ITWJ6NDTF45CBTDO&outputsize=compact
API Response: {'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
Error: No candlestick data found in response.
{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
Failed to fetch data. Retrying in 15 seconds.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a4932569c87b7f2da4.gradio.live


# One-stretch

In [12]:
import requests
import pandas as pd
import plotly.graph_objects as go
from ultralytics import YOLO
import cv2
import time
import gradio as gr

API_KEY = "ITWJ6NDTF45CBTDO"

def get_stock_candlestick_data(symbol, interval="5min", output_size="compact"):
    """
    Fetch stock candlestick data from Alpha Vantage.
    """
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&apikey={API_KEY}&outputsize={output_size}"
    print(f"Fetching data from: {url}")  # Debugging
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        print("API Response:", data)  # Debugging
        if f"Time Series ({interval})" in data:
            return data[f"Time Series ({interval})"]
        else:
            print("Error: No candlestick data found in response.")
            print(data)
            return None
    else:
        print(f"Error fetching data: {response.status_code}")
        print(response.text)
        return None

def process_stock_candlestick_data(data):
    """
    Process Alpha Vantage stock candlestick data into a DataFrame.
    """
    rows = []
    for timestamp, values in data.items():
        rows.append({
            "timestamp": timestamp,
            "open": float(values["1. open"]),
            "high": float(values["2. high"]),
            "low": float(values["3. low"]),
            "close": float(values["4. close"]),
            "volume": float(values["5. volume"])
        })
    return pd.DataFrame(rows)

def generate_candlestick_chart(df, n=50):
    """
    Generate a candlestick chart using Plotly with the last n data points.
    """
    df = df.tail(n)  # Use only the last n rows
    fig = go.Figure(data=[go.Candlestick(
        x=df["timestamp"],
        open=df["open"],
        high=df["high"],
        low=df["low"],
        close=df["close"]
    )])
    fig.update_layout(
        title="Candlestick Chart",
        xaxis_title="Time",
        yaxis_title="Price",
        xaxis_rangeslider_visible=False
    )
    # Removed fig.show() since Gradio will display the image
    fig.write_image("candlestick.png")

def yolo_model(img_path, model):
    """
    Run YOLO model on the image and count GAP UP and GAP DOWN patterns.
    """
    results = model(img_path)
    gap_up_count = 0
    gap_down_count = 0
    for result in results:
        classes = result.boxes.cls
        for cls in classes:
            if cls == 0:
                gap_down_count += 1
            elif cls == 1:
                gap_up_count += 1
        annotated_image = result.plot()
    return annotated_image, gap_up_count, gap_down_count

def detect_gap_patterns(symbol):
    """
    Main function to fetch data, generate charts, and detect GAP patterns in near-real-time.
    """
    model = YOLO("/content/best.pt")  # Load model once outside the loop
    while True:
        data = get_stock_candlestick_data(symbol)
        if not data:
            print("Failed to fetch data. Retrying in 15 seconds.")
            time.sleep(15)
            continue  # Retry instead of exiting

        df = process_stock_candlestick_data(data)
        generate_candlestick_chart(df, n=50)  # Generate chart with last 50 candles
        annotated_image, gap_up_count, gap_down_count = yolo_model("candlestick.png", model)
        cv2.imwrite("annotated_output.png", annotated_image)
        yield "annotated_output.png", gap_up_count, gap_down_count
        time.sleep(15)  # Wait 15 seconds to respect API rate limits

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# GAP Pattern Detection in Real-Time Stock Charts")
    gr.Markdown("Enter a stock symbol (e.g., AAPL) to detect GAP UP and GAP DOWN patterns in near-real-time candlestick charts.")

    with gr.Row():
        symbol_input = gr.Textbox(label="Stock Symbol", placeholder="Enter a stock symbol (e.g., AAPL)")
        submit_button = gr.Button("Start Real-Time Detection")

    with gr.Row():
        output_image = gr.Image(label="Annotated Candlestick Chart")
        gap_up_output = gr.Textbox(label="GAP UP Count")
        gap_down_output = gr.Textbox(label="GAP DOWN Count")

    # Start real-time detection when the button is clicked
    submit_button.click(
        fn=detect_gap_patterns,
        inputs=symbol_input,
        outputs=[output_image, gap_up_output, gap_down_output]
    )

# Launch the Gradio app
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f52e788ac566131ba5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Fetching data from: https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=AAPL&interval=5min&apikey=ITWJ6NDTF45CBTDO&outputsize=compact
API Response: {'Meta Data': {'1. Information': 'Intraday (5min) open, high, low, close prices and volume', '2. Symbol': 'AAPL', '3. Last Refreshed': '2025-02-24 19:55:00', '4. Interval': '5min', '5. Output Size': 'Compact', '6. Time Zone': 'US/Eastern'}, 'Time Series (5min)': {'2025-02-24 19:55:00': {'1. open': '248.0600', '2. high': '248.2500', '3. low': '248.0500', '4. close': '248.0600', '5. volume': '3233'}, '2025-02-24 19:50:00': {'1. open': '248.2100', '2. high': '248.3000', '3. low': '248.0512', '4. close': '248.0600', '5. volume': '2154'}, '2025-02-24 19:45:00': {'1. open': '248.2470', '2. high': '248.2962', '3. low': '248.1300', '4. close': '248.2500', '5. volume': '1625'}, '2025-02-24 19:40:00': {'1. open': '248.2100', '2. high': '248.3000', '3. low': '248.1000', '4. close': '248.3000', '5. volume': '1383'}, '2025-02-24 19:35